In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# !pip install konlpy

In [ ]:
import re
import os
import unicodedata
import urllib3
import zipfile
import shutil
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
# from konlpy.tag import Okt, Komoran, Mecab # komoran, hannanum, kkma, mecab
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

## Download the data


In [ ]:
# !gdown --id 1--vjTQGnQtGVU-IMZ1SEEhU80gYgZm8P

In [ ]:
# df = pd.read_csv('/content/구어체.csv')
# df = df.sample(frac=1).reset_index(drop=True)
# df

## Configuration


In [ ]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
hidden_units = 256  # Latent dimensionality of the encoding space.
embedding_dim = 64
num_samples = 300000  # Number of samples to train on.

## Prepare the data


In [ ]:
# def preprocess_sentence(sent):
#   # 단어와 구두점 사이에 공백을 만듭니다.
#   # Ex) "he is a boy." => "he is a boy ."
#   sent = re.sub(r"([?.!,¿])", r" \1", sent)

#   # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환합니다.
#   sent = re.sub(r"[^a-zA-Zㄱ-ㅎ가-힣!.?]+", r" ", sent)

#   # 다수 개의 공백을 하나의 공백으로 치환
#   sent = re.sub(r"\s+", " ", sent)
#   return sent

In [ ]:
# def load_preprocessed_data():

#     encoder_input, decoder_input, decoder_target = [], [], []
#     okt = Okt()

#     for i in range(int(len(df))):
#         # source 데이터와 target 데이터 분리
#         src_line, tar_line = df['번역문'].iloc[i], df['원문'].iloc[i]
#         # source 데이터 전처리
#         src_line = [w for w in preprocess_sentence(src_line).split()]
#         src_line.reverse()
#         # target 데이터 전처리
#         tar_line = okt.morphs(tar_line)
#         tar_line_in = ['<sos>'] + tar_line  # [w for w in ("<sos> " + tar_line).split()]  # teacher forcing을 위한 정답셋 
#         tar_line_out = tar_line + ['<eos>'] # [w for w in (tar_line + " <eos>").split()]
#         encoder_input.append(src_line)
#         decoder_input.append(tar_line_in)
#         decoder_target.append(tar_line_out)    
#         if i == num_samples - 1:
#             break

#     return encoder_input, decoder_input, decoder_target

In [ ]:
# # 전처리 테스트
# en_sent = u"While the defense minister and the DSC (Defense Security Command) chief made conflicting statements, there was an unprecedented situation in which the defense minister dismissed the exposes remarks made by the incumbent DSC chief as lies."
# kr_sent = u"국방장관과 기무사령관이 서로 엇갈리는 진술을 내놓는가 하면, 현역 기무부대장의 폭로성 발언을 국방장관이 거짓말이라고 공박하는 전례없는 상황이 벌어졌다."

# print('전처리 전 영어 문장 :', en_sent)
# print('전처리 후 영어 문장 :',preprocess_sentence(en_sent))
# print('전처리 전 한국어 문장 :', kr_sent)
# print('전처리 후 한국어어 문장 :', preprocess_sentence(kr_sent))

In [ ]:
# sents_en_in, sents_kor_in, sents_kor_out  = load_preprocessed_data()

In [ ]:
# ## Save pickle
# with open("sents_en_in.pickle","wb") as fw:
#     pickle.dump(sents_en_in, fw)
# with open("sents_kor_in.pickle","wb") as fw:
#     pickle.dump(sents_kor_in, fw)
# with open("sents_kor_out.pickle","wb") as fw:
#     pickle.dump(sents_kor_out, fw)

In [ ]:
!gdown 1lwf3670ldheQCZqS68y2eYqflQBUJULz
!gdown 1TP5zHiFzOv1XirVRAkTMcLkwZf3rQKU4
!gdown 1bUukSpLVfLKcwjFHCzIPNeXbexj4QILh

Downloading...
From: https://drive.google.com/uc?id=1lwf3670ldheQCZqS68y2eYqflQBUJULz
To: /content/sents_en_in.pickle
100% 47.4M/47.4M [00:00<00:00, 230MB/s]
Downloading...
From: https://drive.google.com/uc?id=1TP5zHiFzOv1XirVRAkTMcLkwZf3rQKU4
To: /content/sents_kor_in.pickle
100% 56.8M/56.8M [00:00<00:00, 239MB/s]
Downloading...
From: https://drive.google.com/uc?id=1bUukSpLVfLKcwjFHCzIPNeXbexj4QILh
To: /content/sents_kor_out.pickle
100% 56.8M/56.8M [00:00<00:00, 188MB/s]


In [ ]:
## Load pickle
with open("/content/sents_en_in.pickle","rb") as fr:
    sents_en_in = pickle.load(fr)
with open("/content/sents_kor_in.pickle","rb") as fr:
    sents_kor_in = pickle.load(fr)
with open("/content/sents_kor_out.pickle","rb") as fr:
    sents_kor_out = pickle.load(fr)

In [ ]:
sents_en_in = sents_en_in[0:num_samples]
sents_kor_in = sents_kor_in[0:num_samples]
sents_kor_out = sents_kor_out[0:num_samples]

In [ ]:
i = 999
print('인코더의 입력 :',sents_en_in[i])
print('디코더의 입력 :',sents_kor_in[i])
print('디코더의 레이블 :',sents_kor_out[i])

인코더의 입력 : ['.', 'desperately', 'English', 'need', 'not', 'does', 'life', 'My']
디코더의 입력 : ['<sos>', '제', '삶은', '영어', '가', '절실', '히', '필요하지', '않습니다', '.']
디코더의 레이블 : ['제', '삶은', '영어', '가', '절실', '히', '필요하지', '않습니다', '.', '<eos>']


단어집합을 생성하겠습니다. 

In [ ]:
# 영어

tokenizer_en = Tokenizer(filters="", lower = False)
tokenizer_en.fit_on_texts(sents_en_in)
encoder_input = tokenizer_en.texts_to_sequences(sents_en_in)

In [ ]:
# 한국어
tokenizer_kor = Tokenizer(filters='', lower = False)
tokenizer_kor.fit_on_texts(sents_kor_in)
tokenizer_kor.fit_on_texts(sents_kor_out)

decoder_input = tokenizer_kor.texts_to_sequences(sents_kor_in)
decoder_target = tokenizer_kor.texts_to_sequences(sents_kor_out)

각 문장을 입력층의 최대 길이만큼 패딩(padding)을 수행합니다. 

In [ ]:
encoder_input = pad_sequences(encoder_input, padding='post')
decoder_input = pad_sequences(decoder_input, padding='post')
decoder_target = pad_sequences(decoder_target, padding='post')

In [ ]:
print('인코더의 입력의 크기(shape) :',encoder_input.shape)
print('디코더의 입력의 크기(shape) :',decoder_input.shape)
print('디코더의 레이블의 크기(shape) :',decoder_target.shape)

인코더의 입력의 크기(shape) : (300000, 51)
디코더의 입력의 크기(shape) : (300000, 48)
디코더의 레이블의 크기(shape) : (300000, 48)


샘플은 총 33,000개 존재하며 영어 문장의 길이는 8, 프랑스어 문장의 길이는 16입니다. 단어 집합의 크기를 정의합니다.


In [ ]:
print(tokenizer_en.word_index)
print(tokenizer_kor.word_index)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
src_vocab_size = len(tokenizer_en.word_index) + 1
tar_vocab_size = len(tokenizer_kor.word_index) + 1

print(f"영어 단어 집합의 크기 : {src_vocab_size}, 한국어 단어 집합의 크기 : {tar_vocab_size}")

영어 단어 집합의 크기 : 55244, 한국어 단어 집합의 크기 : 92137


단어로부터 정수를 얻는 딕셔너리와 정수로부터 단어를 얻는 딕셔너리를 각각 만들어줍니다. 이들은 훈련을 마치고 예측값과 실제값을 비교하는 단계에서 사용합니다. 

In [ ]:
src_to_index = tokenizer_en.word_index  
index_to_src = tokenizer_en.index_word  

tar_to_index = tokenizer_kor.word_index 
index_to_tar = tokenizer_kor.index_word 

In [ ]:
print(src_to_index)
print(index_to_src)
print(tar_to_index)
print(index_to_tar)

{'.': 1, 'the': 2, 'I': 3, 'to': 4, 'a': 5, 'you': 6, 'is': 7, 'of': 8, 'and': 9, 'in': 10, '?': 11, 'for': 12, 'it': 13, 'that': 14, 'with': 15, 'have': 16, 'my': 17, 'are': 18, 's': 19, 'be': 20, 'The': 21, 't': 22, 'on': 23, 'will': 24, 'this': 25, 'me': 26, 'can': 27, 'your': 28, 'was': 29, 'It': 30, 'not': 31, 'as': 32, 'at': 33, 'we': 34, 'We': 35, 'like': 36, 'from': 37, 'so': 38, 'time': 39, 'm': 40, 'do': 41, 'by': 42, 'about': 43, 'want': 44, 'an': 45, 'but': 46, 'if': 47, 'there': 48, 'when': 49, 'You': 50, 'am': 51, 'because': 52, 'Please': 53, 'go': 54, 'has': 55, 'know': 56, 'This': 57, 'get': 58, 'think': 59, 'don': 60, 'If': 61, 'would': 62, 'or': 63, 'one': 64, 'all': 65, 'good': 66, 'more': 67, 'people': 68, 'our': 69, 'after': 70, 'who': 71, 'up': 72, 'Korea': 73, 'now': 74, 'out': 75, 'make': 76, 'work': 77, 'There': 78, 'What': 79, 'going': 80, 'My': 81, 'been': 82, 'should': 83, 'He': 84, 'what': 85, 'they': 86, 'need': 87, 'today': 88, 'll': 89, 'send': 90, 'he':

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



테스트 데이터를 분리하겠습니다. 인코더 input, 디코더 input, 디코더 output이 동일하게 분리되어야 하기 때문에 랜덤한 정수배열을 만들어 직접 나누겠습니다. 

우선 랜덤하게 시퀀스를 섞습니다. 

In [ ]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print('랜덤 시퀀스 :',indices)

랜덤 시퀀스 : [253576 130261 288348 ... 255992 159217  54528]


In [ ]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [ ]:
# decoder_input[indices[0]], decoder_target[indices[0]]

`<sos>` `<eos>` 토큰을 제외한 다른 단어들은 동일한 인덱스를 갖고 있어야 합니다. 

10%의 데이터를 test데이터로 분리하겠습니다. 

In [ ]:
n_of_val = int(num_samples*0.1)

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

In [ ]:
print('훈련 source 데이터의 크기 :',encoder_input_train.shape)
print('훈련 target 데이터의 크기 :',decoder_input_train.shape)
print('훈련 target 레이블의 크기 :',decoder_target_train.shape)
print('테스트 source 데이터의 크기 :',encoder_input_test.shape)
print('테스트 target 데이터의 크기 :',decoder_input_test.shape)
print('테스트 target 레이블의 크기 :',decoder_target_test.shape)

훈련 source 데이터의 크기 : (270000, 51)
훈련 target 데이터의 크기 : (270000, 48)
훈련 target 레이블의 크기 : (270000, 48)
테스트 source 데이터의 크기 : (30000, 51)
테스트 target 데이터의 크기 : (30000, 48)
테스트 target 레이블의 크기 : (30000, 48)


## Model

이제 번역기를 빌드해보겠습니다. 

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

인코더를 설계합니다. LSTM에서 state_h, state_c를 리턴받는데, 이는 각각 RNN 챕터에서 LSTM을 처음 설명할 때 언급하였던 은닉 상태와 셀 상태에 해당됩니다. 이 두 가지 상태를 encoder_states에 저장합니다. encoder_states를 디코더에 전달하므로서 이 두 가지 상태 모두를 디코더로 전달할 예정입니다. 이것이 앞서 배운 컨텍스트 벡터입니다.

In [ ]:
# 인코더
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(src_vocab_size, embedding_dim)(encoder_inputs) # 임베딩 층

###############
encoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True) # 상태값 리턴을 위해 return_state는 True
###############

encoder_outputs, state_h, state_c = encoder_lstm(enc_emb) # 은닉 상태와 셀 상태를 리턴

encoder_states = [state_h, state_c] # 인코더의 은닉 상태와 셀 상태를 저장

이번에는 Attention을 추가합니다. 
- Attention 중 가장 기초적인 dot-product attention을 예제로 진행하겠습니다. 
1. attention score 계산
2. softmax로 attention distribution 계산
3. 각 인코더의 어텐션 가중치와 은닉 상태를 가중합하여 어텐션 값 계산
4. 어텐션값과 디코더의 t시점의 은닉상태를 연결
5. 출력층 연산의 입력이 되는 st 계산

디코더에서는 attention layer를 추가합니다. 
- 이때, s_는 은닉상태와 디코더의 최종 출력은 연결해야 하므로, shape을 맞춰주기 위하여 축을 추가합니다. 
- attention layer는 디코더의 은닉상태와 인코더의 은닉상태 전체를 받아 컨텍스트 벡터를 생성합니다. 
- 마지막으로 생성한 컨텍스트 벡터와 디코더의 은닉상태 전체를 softmax layer에 넣고 인덱스를 예측합니다. 


In [ ]:
from keras.layers import AdditiveAttention, Dense 

class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units, **kwargs):
      super().__init__()
      self.units = units
      self.W1 = Dense(units, use_bias=False)
      self.W2 = Dense(units, use_bias=False)

      self.attention = AdditiveAttention()

  def call(self, query, value):
      w1_query = self.W1(query)
      w2_key = self.W2(value)
      context_vector, attention_weights = self.attention(inputs = [w1_query, value, w2_key], return_attention_scores = True)

      return context_vector, attention_weights
  def get_config(self):
      config = super().get_config()
      config.update({
          "units": self.units,
      })
      return config


In [ ]:
# 디코더
decoder_inputs = Input(shape=(None,))

# 임베딩 층
dec_emb_layer = Embedding(tar_vocab_size, hidden_units)

# 임베딩 결과
dec_emb = dec_emb_layer(decoder_inputs) 

######################
# 상태값 리턴을 위해 return_state는 True, 모든 시점에 대해서 단어를 예측하기 위해 return_sequences는 True
decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True) 
######################

# 인코더의 은닉 상태를 초기 은닉 상태(initial_state)로 사용
decoder_outputs, _ , _ = decoder_lstm(dec_emb, initial_state=encoder_states)

######################
# attention
S_ = tf.concat([state_h[:, tf.newaxis, :], decoder_outputs[:, :-1, :]], axis=1)

attention = BahdanauAttention(hidden_units)
context_vector, _ = attention(S_, encoder_outputs)

concat = tf.concat([decoder_outputs, context_vector], axis=-1)

# 모든 시점의 결과에 대해서 소프트맥스 함수를 사용한 출력층을 통해 단어 예측
decoder_dense = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(concat)

######################

In [ ]:
# 모델의 입력과 출력을 정의.
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [ ]:
cp_path = '/content/drive/MyDrive/EN2KO Translator (1-Layered LSTM Model with Reversed Sequence and Attention)/30만개'
# cp_path = '/content'

mc = ModelCheckpoint(filepath = cp_path + '/best_model.h5', verbose=1, save_best_only = True)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)

model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, 
          validation_data=([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=64, epochs=100, callbacks=[mc, es])

Epoch 1/100
  23/4219 [..............................] - ETA: 29:05 - loss: 7.1400 - acc: 0.6891

KeyboardInterrupt: ignored

In [ ]:
!gdown 1i_EvYrS6FmTHYkyjluby11o5v5OW5nAh

Downloading...
From: https://drive.google.com/uc?id=1i_EvYrS6FmTHYkyjluby11o5v5OW5nAh
To: /content/best_model.h5
100% 905M/905M [00:05<00:00, 176MB/s]


In [ ]:
model.load_weights('/content/best_model.h5')

In [ ]:
# model.summary()

In [ ]:
# tf.keras.utils.plot_model(model, to_file='model.png', )

## 훈련된 모델로 번역하기

* 평가 함수는 여기서 *교사 강요(teacher forcing)*를 사용하기 못하는 것을 제외하고는 훈련 루프와 비슷합니다. 각 마지막 시점(time step)에서 이전 디코더 인코더의 결과와 은닉 상태(hidden state)를 가진 예측 값을 디코더에 입력합니다.

    1. 번역하고자 하는 입력 문장이 인코더로 입력되어 인코더의 마지막 시점의 은닉 상태와 셀 상태를 얻습니다.
    2. 인코더의 은닉 상태와 셀 상태, 그리고 토큰 <sos>를 디코더로 보냅니다.
    3. 디코더가 토큰 <eos>가 나올 때까지 다음 단어를 예측하는 행동을 반복합니다.

In [ ]:
# 인코더 - 훈련과 동일 

######################
encoder_model = Model(encoder_inputs, [encoder_outputs, encoder_states])
######################

encoder_model.compile()
encoder_model.save(cp_path + '/encoder.h5')

In [ ]:
# 디코더 


# 이전 시점의 상태를 보관할 텐서
decoder_state_input_h = Input(shape=(hidden_units,))
decoder_state_input_c = Input(shape=(hidden_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

######################
encoder_state_h = Input(shape=(hidden_units,))
encoder_outputs2 = Input(shape =(None, hidden_units,))
######################

# 훈련 때 사용했던 임베딩 층을 재사용
dec_emb2 = dec_emb_layer(decoder_inputs)

# 다음 단어 예측을 위해 이전 시점의 상태를 현 시점의 초기 상태로 사용
decoder_outputs2, state_h, state_c = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h, state_c]

######################
S_ = tf.concat([encoder_state_h[:, tf.newaxis, :], decoder_outputs2[:, :-1, :]], axis=1) # query

# Attention
context_vector, att_score = attention(S_, encoder_outputs2)
decoder_concat = tf.concat([decoder_outputs2, context_vector], axis=-1)

# 모든 시점에 대해서 단어 예측 (Fully Connected)
decoder_outputs2 = decoder_dense(decoder_concat)

# 수정된 디코더
decoder_model = Model(
    [decoder_inputs, encoder_state_h, encoder_outputs2] + decoder_states_inputs,
    [decoder_outputs2, att_score] + decoder_states2)

######################

decoder_model.compile()
decoder_model.save(cp_path + '/decoder.h5')

테스트 단계에서의 동작을 위한 decode_sequence 함수를 구현합니다. 
1. 입력 문장이 들어오면 인코더는 마지막 시점까지 전개하여 마지막 시점의 은닉 상태와 셀 상태를 리턴합니다. -> `states_value` 
2. 디코더의 초기 입력으로 <SOS>를 준비합니다. -> `target_seq`
3. 이 두 가지 입력을 가지고 while문 안으로 진입하여 이 두 가지를 디코더의 입력으로 사용합니다. 
4. 이제 디코더는 현재 시점에 대해서 예측합니다. 
    - 예측 벡터로부터 현재 시점의 예측 단어인 target_seq를 얻고, h와 c 이 두 개의 값은 states_value에 저장합니다. 
    - while문의 다음 루프. 즉, 두번째 시점의 디코더의 입력으로 다시 target_seq와 states_value를 사용합니다. 
    - 이를 현재 시점의 예측 단어로 <eos>를 예측하거나 번역 문장의 길이가 50이 넘는 순간까지 반복합니다. 
5. 각 시점마다 번역된 다어는 decoded_sentence에 누적하여 저장하였다가 최종 번역 시퀀스로 리턴합니다.

In [ ]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 마지막 시점의 상태(은닉 상태, 셀 상태)를 얻음
  encoder_test, states_value = encoder_model.predict(input_seq)

  # <SOS>에 해당하는 정수 생성
  target_seq = np.zeros((1,1))
  target_seq[0, 0] = tar_to_index['<sos>']

  stop_condition = False
  decoded_sentence = ''

  # stop_condition이 True가 될 때까지 루프 반복
  # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정합니다.
  while not stop_condition:
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
    output_tokens, att_score, h, c = decoder_model.predict([target_seq, states_value[0], encoder_test] + states_value)


    # 예측 결과를 단어로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 단어를 예측 문장에 추가
    decoded_sentence += ' '+sampled_char

    # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
    if (sampled_char == '<eos>' or
        len(decoded_sentence) > 50):
        stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = sampled_token_index

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return decoded_sentence

결과 확인을 위한 함수를 만듭니다.

In [ ]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_src(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word != 0):
      sentence = sentence + index_to_src[encoded_word] + ' '
  return sentence

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_tar(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word != 0 and encoded_word != tar_to_index['<sos>'] and encoded_word != tar_to_index['<eos>']):
      sentence = sentence + index_to_tar[encoded_word] + ' '
  return sentence

In [ ]:
encoder_input_train[0:1]

array([[    1,  1923,     9,  4295, 10947,    37,  5487, 13935,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0]], dtype=int32)

In [ ]:
encoder_input_train[0]

array([    1,  1923,     9,  4295, 10947,    37,  5487, 13935,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0], dtype=int32)

In [ ]:
encoder_input_train

array([[   1, 1923,    9, ...,    0,    0,    0],
       [   1, 1393,    2, ...,    0,    0,    0],
       [   1,  368,  975, ...,    0,    0,    0],
       ...,
       [   1,  200,   58, ...,    0,    0,    0],
       [   1, 8068,   96, ...,    0,    0,    0],
       [   1,  611,    2, ...,    0,    0,    0]], dtype=int32)

In [ ]:
for seq_index in range(10):
  input_seq = encoder_input_train[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("입력문장 :",seq_to_src(encoder_input_train[seq_index]))
  print("정답문장 :",seq_to_tar(decoder_input_train[seq_index]))
  print("번역문장 :",decoded_sentence[1:-5])
  print("-"*50)

입력문장 : . Singapore and Mexico Bangladesh from stamps Collecting 
정답문장 : 방글라데쉬 , 멕시코 그리고 싱가포르 에서 수집 된 우표 들 . 
번역문장 : 멕시코 멕시코 , 멕시코 , 멕시코 , 멕시코 , 멕시코 , 멕시코 , 멕시코 , 
--------------------------------------------------
입력문장 : . window the open you when ocean blue the see will You 
정답문장 : 당신 이 창문 을 열면 파란 바다 가 보일 거 예요 . 
번역문장 : 창문 을 열면 파란 바다 를 보일 거 예요 . 
--------------------------------------------------
입력문장 : . yet stock of out is product the think t don I 
정답문장 : 아직 재고 가 부족하진 않은 것 같습니다 . 
번역문장 : 저 는 재고 가 없는 제품 이 아닙니다 . 
--------------------------------------------------
입력문장 : . study to late too up stay not Do 
정답문장 : 너무 늦게까지 공부 한다고 깨어 있지 마 . 
번역문장 : 공부 를 너무 늦지 않게 조심하세요 . 
--------------------------------------------------
입력문장 : ? English at good friends your Are 
정답문장 : 당신 의 한국인 친구 들 은 영어 를 잘 합니까 ? 
번역문장 : 당신 의 친구 들 은 영어 를 잘 합니까 ? 
--------------------------------------------------
입력문장 : . year this incidents troubling and sad many had we since things happy and health good with filled 

In [ ]:
for seq_index in range(10):
  input_seq = encoder_input_test[seq_index: seq_index + 1]
  print(input_seq)
  decoded_sentence = decode_sequence(input_seq)

  print("입력문장 :",seq_to_src(encoder_input_test[seq_index]))
  print("정답문장 :",seq_to_tar(decoder_input_test[seq_index]))
  print("번역문장 :",decoded_sentence[1:-5])
  print("-"*50)

[[   1  716   29  605    2    9 1842   29   13 2895  123 3431    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0]]
입력문장 : . clean was air the and quiet was it concerns her Despite 
정답문장 : 소녀 의 걱정 과는 달리 조용하고 공기 도 괜찮았어요 . 
번역문장 : 그녀 의 눈 에 많은 것 을 받았고 , 공기 가 굉장히 깨끗합니다 . 
--------------------------------------------------
[[  11  107  275   64   12   95   26   15 5625    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0]]
입력문장 : ? please night one for just me with Stay 
정답문장 : 하룻밤 만 저 와 같이 있어 줘요 , 제발 요 ? 
번역문장 : 하룻밤 만 더 자고 싶은데 좀 봐주세요 ? 
--------------------------------------------------
[[  808   480     7   239 21088 51030     0     0     0     0     0     0
      0     0     0     

## BLEU Score (Bilingual Evaluation Understudy Score)
기계 번역의 성능이 얼마나 뛰어난가를 측정하기 위해 사용되는 대표적인 방법 중 하나인 BLEU(Bilingual Evaluation Understudy)를 측정해봅시다. 
- 논문 : BLEU: a Method for Automatic Evaluation of Machine Translation
- BLEU 점수는 기계 번역된 텍스트와 고품질 참조 번역 세트의 유사성을 측정하는 0과 1 사이의 숫자입니다. 
    - 값이 0이면 기계 번역된 출력이 참조 번역과 겹치는 부분이 없는 것을 의미하고(저품질)
    - 1이면 참조 번역과 완벽하게 겹치는 것을 의미합니다(고품질)

- BLEU 설명 
    - https://jrc-park.tistory.com/273
    - https://wikidocs.net/31695
    - https://cloud.google.com/translate/automl/docs/evaluate?hl=ko#bleu

In [ ]:
import numpy as np
from collections import Counter
from nltk import ngrams

BLEU는 기계 번역 결과와 사람이 직접 번역한 결과가 얼마나 유사한지 비교하여 번역에 대한 성능을 측정하는 방법입니다. 

In [ ]:
import nltk.translate.bleu_score as bleu

candidate = 'It is a guide to action which ensures that the military always obeys the commands of the party'
references = [
    'It is a guide to action that ensures that the military will forever heed Party commands',
    'It is the guiding principle which guarantees the military forces always being under the command of the Party',
    'It is the practical guide for the army always to heed the directions of the party'
]

print('패키지 NLTK의 BLEU :',bleu.sentence_bleu(list(map(lambda ref: ref.split(), references)),candidate.split()))

패키지 NLTK의 BLEU : 0.5045666840058485


In [ ]:
ref_train = [seq_to_tar(sen) for sen in decoder_input_train[:100]]

In [ ]:
from tqdm.notebook import tqdm 

pred_train = []
for idx in tqdm(range(len(encoder_input_train[:100]))):
    pred_train.append(decode_sequence(encoder_input_train[idx:idx+1]))

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
bleu_score = 0
for ref, pred in zip(ref_train, pred_train):
    if len(ref) == 0 or len(pred) == 0:
        continue
    bleu_score += bleu.sentence_bleu(ref, pred)

bleu_score = bleu_score/len(ref_train)
bleu_score

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.721791013713076

In [ ]:
ref_test = [seq_to_tar(sen) for sen in decoder_input_test[:100]]
pred_test = [decode_sequence(encoder_input_test[idx:idx+1]) for idx in tqdm(range(len(encoder_input_test[:100])))]

bleu_score = 0
for ref, pred in zip(ref_test, pred_test):
    if len(ref) == 0 or len(pred) == 0:
        continue
    bleu_score += bleu.sentence_bleu(ref, pred)

bleu_score = bleu_score/len(ref_test)
bleu_score

  0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.7155876666679784

### Reference

- https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html
- https://wikidocs.net/86900

In [171]:
!gdown 1Jalge4VlMDWGJb_UbUIj7LLD78TO1a7n

Downloading...
From: https://drive.google.com/uc?id=1Jalge4VlMDWGJb_UbUIj7LLD78TO1a7n
To: /content/영한테스트_영어.csv
100% 35.0k/35.0k [00:00<00:00, 43.8MB/s]


In [201]:
data = pd.read_csv('/content/영한테스트_영어.csv')
data

,영어
0,This is good food for many people to eat toget...
1,I didn't check the list yet but more people th...
2,"Sure, I'll get Kimchi and pickles soon."
3,I think we should choose the second one.
4,"Sure, I have a lot of small bills."
...,...
495,"When you are watching a performance, it's impo..."
496,The toner pad with skin on it was really conve...
497,"Yes, I will check the state of the car before ..."
498,"It's too bad you can't join, but you're mother..."


In [202]:
# samples = {
#     'col': [
#              'this is english.',
#              'this is great.',
#              'I am a good man.',
#              'You are a good woman too.',
#              'good morning how are you today?',
#              'what are you doing now?'

#     ]
# }

In [203]:
# data = pd.DataFrame(samples)

In [204]:
def preprocess(sent):
    # 단어와 구두점 사이에 공백을 만듭니다.
    # Ex) "he is a boy." => "he is a boy ."
    sent = re.sub(r"([?.!,¿])", r" \1", sent)

    # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환합니다.
    sent = re.sub(r"[^a-zA-Zㄱ-ㅎ가-힣!.?]+", r" ", sent)

    # 다수 개의 공백을 하나의 공백으로 치환
    sent = re.sub(r"\s+", " ", sent)
    return sent


def load_preprocessed(df, df_col_name):
    
    encoder_input = []

    for i in range(int(len(df))):
        # source 데이터와 target 데이터 분리
        src_line = df[f'{df_col_name}'].iloc[i]
        # source 데이터 전처리
        src_line = [w for w in preprocess(src_line).split()]
        src_line.reverse()

        encoder_input.append(src_line)

    return encoder_input

In [205]:
preprocessed = load_preprocessed(data, '영어')

In [206]:
en_input = tokenizer_en.texts_to_sequences(preprocessed)

In [207]:
input_data = pad_sequences(en_input, padding='post', maxlen=51) # maxlen 51?

In [208]:
input_len = len(input_data)

In [209]:
result = []

# 역순 인풋
for seq_index in range(input_len):
    input_seq = input_data[seq_index: seq_index + 1]
    print(input_seq)
    decoded_sentence = decode_sequence(input_seq)

    print("입력문장 :",seq_to_src(input_data[seq_index]))
    print("번역문장 :",decoded_sentence[1:-5])
    result.append(decoded_sentence[1:-5])
    print("-"*50)

result_dic = {
    '4조' : result
}

dff = pd.DataFrame(result_dic)

with open('/content/drive/MyDrive/EN2KO Translator (1-Layered LSTM Model with Reversed Sequence and Attention)/30만개/4조결과.pickle', 'wb') as f:
    pickle.dump(dff, f, pickle.HIGHEST_PROTOCOL)

[[  1 220 224   4  68 106  12 173  66   7  57   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]]
입력문장 : . together eat to people many for food good is This 
번역문장 : 이 것 은 사람 들 이 음식 을 많이 먹는 좋은 음식 입니다 . 
--------------------------------------------------
[[  1 126  22 186 830   3 113  68  67  46 368 502   2 142  22 186   3   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]]
입력문장 : . come t didn expected I than people more but yet list the check t didn I 
번역문장 : 아직 체크 목록 을 확인 해보니 아직 보다 더 많은 것 을 못 했어요 . 
--------------------------------------------------
[[    1   232 12947     9  1884    58    89     3  4966     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0    

In [191]:
# 정순 인풋
for seq_index in range(input_len):
    input_seq = input_data[seq_index: seq_index + 1]
    print(input_seq)
    decoded_sentence = decode_sequence(input_seq)

    print("입력문장 :",seq_to_src(input_data[seq_index]))
    print("번역문장 :",decoded_sentence[1:-5])
    print("-"*50)

[[ 57   7  66 173  12 106  68   4 224 220   1   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]]
입력문장 : This is good food for many people to eat together . 
번역문장 : 이렇게 많은 음식 을 먹으면서 많은 사람 이 있습니다 . 
--------------------------------------------------
[[  3 186  22 142   2 502 368  46  67  68 113   3 830 186  22 126   1   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]]
입력문장 : I didn t check the list yet but more people than I expected didn t come . 
번역문장 : 비록 내 가 생각 한 사람 은 아직 사람 들 이 아직 못 한 것 을 알 고 있었지만
--------------------------------------------------
[[ 4966     3    89    58  1884     9 12947   232     1     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0   

In [ ]:
!gdown 1Jalge4VlMDWGJb_UbUIj7LLD78TO1a7n

Downloading...
From: https://drive.google.com/uc?id=1Jalge4VlMDWGJb_UbUIj7LLD78TO1a7n
To: /content/영한테스트_영어.csv
100% 35.0k/35.0k [00:00<00:00, 43.8MB/s]


In [ ]:
data = pd.read_csv('/content/영한테스트_영어.csv')
data

,영어
0,This is good food for many people to eat toget...
1,I didn't check the list yet but more people th...
2,"Sure, I'll get Kimchi and pickles soon."
3,I think we should choose the second one.
4,"Sure, I have a lot of small bills."
...,...
495,"When you are watching a performance, it's impo..."
496,The toner pad with skin on it was really conve...
497,"Yes, I will check the state of the car before ..."
498,"It's too bad you can't join, but you're mother..."


In [212]:
final = pd.concat([data, dff], axis=1)

In [213]:
with open('/content/drive/MyDrive/EN2KO Translator (1-Layered LSTM Model with Reversed Sequence and Attention)/30만개/final.pickle', 'wb') as f:
    pickle.dump(final, f, pickle.HIGHEST_PROTOCOL)

In [214]:
final.to_csv('/content/drive/MyDrive/EN2KO Translator (1-Layered LSTM Model with Reversed Sequence and Attention)/30만개/final.csv', index_label=False)